In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [4]:
df_1 = pd.read_csv(r'C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\clean\df_merged.csv')
df_2 = pd.read_csv(r'C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\raw\df_final_web_data_pt_1.txt')
df_3 = pd.read_csv(r'C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\raw\df_final_web_data_pt_2.txt')
merged_df = pd.concat([df_2, df_3], axis=0)

df_merged = df_1.merge(merged_df, on='client_id', how='inner')
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])
df_merged

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gender,num_accts,balance,calls_6_mnth,logons_6_mnth,variation,visitor_id,visit_id,process_step,date_time
0,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:51:13
1,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:47:50
2,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:46:45
3,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_3,2017-04-02 11:23:08
4,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321182,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,step_2,2017-03-30 23:59:15
321183,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,step_1,2017-03-30 23:58:51
321184,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,start,2017-03-30 23:58:40
321185,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,start,2017-03-30 23:55:11


In [15]:
import pandas as pd

# Step 1: Check the column names to verify 'process_step', 'variation', 'visit_id', and 'date_time' exist
print(df_merged.columns)

# Step 2: Split the dataset into Control and Test groups based on the 'variation' column
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Step 3: Convert 'date_time' to datetime format if it is not already
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Step 4: Define completion: A task is completed if there is a valid 'date_time' for that 'visit_id' and 'process_step'
# We assume a task is considered completed if the 'date_time' is non-null for a given visit_id
control_group['completed'] = control_group['date_time'].notnull().astype(int)
test_group['completed'] = test_group['date_time'].notnull().astype(int)

# Step 5: Calculate completion rates for each process_step in Control group
control_completion_rate_by_step = control_group.groupby('process_step')['completed'].mean().reset_index()

# Step 6: Calculate completion rates for each process_step in Test group
test_completion_rate_by_step = test_group.groupby('process_step')['completed'].mean().reset_index()

# Step 7: Merge the two dataframes to compare completion rates between Control and Test
completion_rates = pd.merge(control_completion_rate_by_step, test_completion_rate_by_step, 
                            on='process_step', 
                            suffixes=('_control', '_test'))

# Step 8: Display the table
print(completion_rates)

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gender',
       'num_accts', 'balance', 'calls_6_mnth', 'logons_6_mnth', 'variation',
       'visitor_id', 'visit_id', 'process_step', 'date_time'],
      dtype='object')
  process_step  completed_control  completed_test
0      confirm                1.0             1.0
1        start                1.0             1.0
2       step_1                1.0             1.0
3       step_2                1.0             1.0
4       step_3                1.0             1.0


C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\3447386494.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_group['completed'] = control_group['date_time'].notnull().astype(int)
C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\3447386494.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_group['completed'] = test_group['date_time'].notnull().astype(int)


In [17]:
import pandas as pd
import scipy.stats as stats

# Step 1: Check the column names to verify 'process_step' and 'variation' exist
print(df_merged.columns)

# Step 2: Split the dataset into Control and Test groups based on the 'variation' column
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Step 3: Define the completion condition based on 'process_step' column
# Assuming 'process_step == 5' means completion
control_group['completed'] = (control_group['process_step'] == 5).astype(int)
test_group['completed'] = (test_group['process_step'] == 5).astype(int)

# Step 4: Calculate the completion rate for each group
completion_rate_control = control_group['completed'].mean()
completion_rate_test = test_group['completed'].mean()

# Step 5: Calculate sample sizes
n_control = len(control_group)
n_test = len(test_group)

# Step 6: Calculate pooled proportion
p_pool = (control_group['completed'].sum() + test_group['completed'].sum()) / (n_control + n_test)

# Step 7: Calculate the z-statistic
z = (completion_rate_control - completion_rate_test) / (p_pool * (1 - p_pool) * (1/n_control + 1/n_test))**0.5

# Step 8: Calculate the p-value
p_value = 2 * (1 - stats.norm.cdf(abs(z)))

# Step 9: Determine statistical significance
alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in completion rates between Control and Test.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in completion rates between Control and Test.")

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gender',
       'num_accts', 'balance', 'calls_6_mnth', 'logons_6_mnth', 'variation',
       'visitor_id', 'visit_id', 'process_step', 'date_time'],
      dtype='object')
Fail to reject the null hypothesis: There is no significant difference in completion rates between Control and Test.


C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\563630013.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_group['completed'] = (control_group['process_step'] == 5).astype(int)
C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\563630013.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_group['completed'] = (test_group['process_step'] == 5).astype(int)
C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\563630013.py:28: RuntimeWarning: invalid value encountered in scalar divide
  z 

In [20]:
import pandas as pd

# Step 1: Check the column names to verify 'process_step', 'variation', 'visit_id', and 'date_time' exist
print(df_merged.columns)

# Step 2: Split the dataset into Control and Test groups based on the 'variation' column
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Step 3: Convert 'date_time' to datetime format if it is not already
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Step 4: Filter and calculate the total completion time for Control group
control_group_sorted = control_group.sort_values(by=['visit_id', 'date_time'])

# For each visit_id, get the first date_time (start) and last date_time (completion)
control_completion_time = control_group_sorted.groupby('visit_id').agg(
    start_time=('date_time', 'min'),
    end_time=('date_time', 'max')
).reset_index()

# Calculate the total completion time for each visit_id
control_completion_time['total_completion_time'] = (control_completion_time['end_time'] - control_completion_time['start_time']).dt.total_seconds()

# Step 5: Calculate the total completion time for all visits in Control group
total_control_time = control_completion_time['total_completion_time'].sum()

# Step 6: Filter and calculate the total completion time for Test group
test_group_sorted = test_group.sort_values(by=['visit_id', 'date_time'])

# For each visit_id, get the first date_time (start) and last date_time (completion)
test_completion_time = test_group_sorted.groupby('visit_id').agg(
    start_time=('date_time', 'min'),
    end_time=('date_time', 'max')
).reset_index()

# Calculate the total completion time for each visit_id
test_completion_time['total_completion_time'] = (test_completion_time['end_time'] - test_completion_time['start_time']).dt.total_seconds()

# Step 7: Calculate the total completion time for all visits in Test group
total_test_time = test_completion_time['total_completion_time'].sum()

# Step 8: Display results
print("Control Group Total Completion Time:", total_control_time, "seconds")
print("Test Group Total Completion Time:", total_test_time, "seconds")

# Step 9: Display the tables for Control and Test groups
print("\nControl Group Completion Time Table:")
print(control_completion_time)

print("\nTest Group Completion Time Table:")
print(test_completion_time)

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gender',
       'num_accts', 'balance', 'calls_6_mnth', 'logons_6_mnth', 'variation',
       'visitor_id', 'visit_id', 'process_step', 'date_time'],
      dtype='object')
Control Group Total Completion Time: 9061334.0 seconds
Test Group Total Completion Time: 11761430.0 seconds

Control Group Completion Time Table:
                           visit_id          start_time            end_time  \
0      100030127_47967100085_936361 2017-03-22 11:07:49 2017-03-22 11:07:49   
1      100037962_47432393712_705583 2017-04-14 16:41:51 2017-04-14 16:44:03   
2      100057941_88477660212_944512 2017-04-09 11:30:10 2017-04-09 11:34:24   
3       10006594_66157970412_679648 2017-04-13 11:50:18 2017-04-13 11:56:12   
4      100071743_53464757454_616703 2017-04-07 10:50:20 2017-04-07 10:50:20   
...                             ...                 ...                 ...   
32176  999859408_41720215615_938916 2017-04-05 13:34:02 201

In [25]:
import pandas as pd

# Step 1: Ensure 'date_time' is in datetime format
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Step 2: Filter by Control and Test groups
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Step 3: Sort the data and calculate total completion time for each visit_id
control_group_sorted = control_group.sort_values(by=['visit_id', 'date_time'])
test_group_sorted = test_group.sort_values(by=['visit_id', 'date_time'])

# For Control group
control_completion_time = control_group_sorted.groupby('visit_id').agg(
    start_time=('date_time', 'min'),
    end_time=('date_time', 'max')
).reset_index()

control_completion_time['total_completion_time'] = (control_completion_time['end_time'] - control_completion_time['start_time']).dt.total_seconds()

# For Test group
test_completion_time = test_group_sorted.groupby('visit_id').agg(
    start_time=('date_time', 'min'),
    end_time=('date_time', 'max')
).reset_index()

test_completion_time['total_completion_time'] = (test_completion_time['end_time'] - test_completion_time['start_time']).dt.total_seconds()

# Step 4: Descriptive statistics for the entire dataset (total completion time)
total_completion_time = pd.concat([control_completion_time['total_completion_time'], test_completion_time['total_completion_time']])

# Descriptive statistics for total
total_stats = total_completion_time.describe()

# Step 5: Descriptive statistics for Control group
control_stats = control_completion_time['total_completion_time'].describe()

# Step 6: Descriptive statistics for Test group
test_stats = test_completion_time['total_completion_time'].describe()

# Step 7: Function to convert seconds to time format (HH:MM:SS)
def seconds_to_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Step 8: Convert the statistics to time format
total_stats_time = total_stats.apply(seconds_to_time)
control_stats_time = control_stats.apply(seconds_to_time)
test_stats_time = test_stats.apply(seconds_to_time)

# Step 9: Print the tables with time format
print("Descriptive Statistics for Total Completion Time (in Time Format):")
print(total_stats_time)

print("\nDescriptive Statistics for Control Group Completion Time (in Time Format):")
print(control_stats_time)

print("\nDescriptive Statistics for Test Group Completion Time (in Time Format):")
print(test_stats_time)

Descriptive Statistics for Total Completion Time (in Time Format):
count    19:15:01
mean     00:05:00
std      00:09:23
min      00:00:00
25%      00:00:35
50%      00:02:45
75%      00:05:49
max      11:39:25
Name: total_completion_time, dtype: object

Descriptive Statistics for Control Group Completion Time (in Time Format):
count    08:56:21
mean     00:04:41
std      00:08:51
min      00:00:00
25%      00:00:22
50%      00:02:40
75%      00:05:41
max      11:39:25
Name: total_completion_time, dtype: object

Descriptive Statistics for Test Group Completion Time (in Time Format):
count    10:18:40
mean     00:05:16
std      00:09:50
min      00:00:00
25%      00:00:51
50%      00:02:48
75%      00:05:56
max      08:47:04
Name: total_completion_time, dtype: object


In [27]:
import pandas as pd

# Step 1: Ensure 'date_time' is in datetime format
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Step 2: Sort the data by visit_id and date_time to calculate completion time correctly
df_merged_sorted = df_merged.sort_values(by=['visit_id', 'date_time'])

# Step 3: Group by process_step and calculate total completion time for each visit_id
df_merged_sorted['start_time'] = df_merged_sorted.groupby('visit_id')['date_time'].transform('min')
df_merged_sorted['end_time'] = df_merged_sorted.groupby('visit_id')['date_time'].transform('max')

# Calculate total completion time in seconds
df_merged_sorted['total_completion_time'] = (df_merged_sorted['end_time'] - df_merged_sorted['start_time']).dt.total_seconds()

# Step 4: Descriptive statistics for each process_step
grouped_stats = df_merged_sorted.groupby('process_step')['total_completion_time'].describe()

# Step 5: Convert the statistics from seconds to time format (HH:MM:SS)
def seconds_to_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Convert the statistics to time format
grouped_stats_time = grouped_stats.applymap(seconds_to_time)

# Step 6: Print the descriptive statistics for each process_step
print("Descriptive Statistics for Completion Time by Process Step (in Time Format):")
print(grouped_stats_time)

Descriptive Statistics for Completion Time by Process Step (in Time Format):
                 count      mean       std       min       25%       50%  \
process_step                                                               
confirm       12:00:14  00:07:32  00:12:43  00:00:00  00:02:21  00:04:16   
start         28:53:56  00:07:40  00:14:46  00:00:00  00:01:31  00:03:54   
step_1        19:00:07  00:08:00  00:12:49  00:00:00  00:02:20  00:04:31   
step_2        15:47:35  00:08:17  00:12:23  00:00:00  00:02:46  00:04:51   
step_3        13:31:15  00:08:25  00:11:38  00:00:00  00:02:54  00:04:58   

                   75%       max  
process_step                      
confirm       00:08:03  05:00:15  
start         00:08:31  11:39:25  
step_1        00:08:58  11:39:25  
step_2        00:09:14  11:39:25  
step_3        00:09:21  05:00:15  


C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\2564579314.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  grouped_stats_time = grouped_stats.applymap(seconds_to_time)


In [29]:
import pandas as pd

# Ensure 'date_time' is in datetime format
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Filter for Control and Test groups
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Function to calculate total completion time and descriptive statistics by process_step
def calculate_completion_stats(group):
    # Sort the data by visit_id and date_time
    group_sorted = group.sort_values(by=['visit_id', 'date_time'])

    # Calculate start and end times per visit_id
    group_sorted['start_time'] = group_sorted.groupby('visit_id')['date_time'].transform('min')
    group_sorted['end_time'] = group_sorted.groupby('visit_id')['date_time'].transform('max')

    # Calculate total completion time in seconds
    group_sorted['total_completion_time'] = (group_sorted['end_time'] - group_sorted['start_time']).dt.total_seconds()

    # Descriptive statistics for each process_step
    grouped_stats = group_sorted.groupby('process_step')['total_completion_time'].describe()

    # Convert the statistics from seconds to time format (HH:MM:SS)
    def seconds_to_time(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        seconds = int(seconds % 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

    # Apply time conversion to the statistics
    grouped_stats_time = grouped_stats.applymap(seconds_to_time)
    
    return grouped_stats_time

# Calculate and print the completion stats for Control group
print("Descriptive Statistics for Completion Time by Process Step (Control Group, in Time Format):")
control_stats = calculate_completion_stats(control_group)
print(control_stats)

# Calculate and print the completion stats for Test group
print("\nDescriptive Statistics for Completion Time by Process Step (Test Group, in Time Format):")
test_stats = calculate_completion_stats(test_group)
print(test_stats)

Descriptive Statistics for Completion Time by Process Step (Control Group, in Time Format):


C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\2157732644.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  grouped_stats_time = grouped_stats.applymap(seconds_to_time)


                 count      mean       std       min       25%       50%  \
process_step                                                               
confirm       04:51:38  00:07:09  00:09:33  00:00:00  00:02:47  00:04:38   
start         13:13:09  00:06:56  00:14:19  00:00:00  00:00:55  00:03:30   
step_1        08:14:17  00:07:21  00:12:10  00:00:00  00:02:14  00:04:22   
step_2        07:11:17  00:07:39  00:10:47  00:00:00  00:02:50  00:04:50   
step_3        06:19:47  00:07:55  00:09:38  00:00:00  00:03:02  00:05:03   

                   75%       max  
process_step                      
confirm       00:08:05  04:44:32  
start         00:07:54  11:39:25  
step_1        00:08:20  11:39:25  
step_2        00:08:47  11:39:25  
step_3        00:09:05  04:44:32  

Descriptive Statistics for Completion Time by Process Step (Test Group, in Time Format):
                 count      mean       std       min       25%       50%  \
process_step                                            

C:\Users\Cecilia\AppData\Local\Temp\ipykernel_25664\2157732644.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  grouped_stats_time = grouped_stats.applymap(seconds_to_time)


In [31]:
import pandas as pd
import scipy.stats as stats

# Ensure 'date_time' is in datetime format
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Filter for Control and Test groups
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Step 1: Calculate total completion time for each group
def calculate_total_completion_time(group):
    group_sorted = group.sort_values(by=['visit_id', 'date_time'])
    group_sorted['start_time'] = group_sorted.groupby('visit_id')['date_time'].transform('min')
    group_sorted['end_time'] = group_sorted.groupby('visit_id')['date_time'].transform('max')
    group_sorted['total_completion_time'] = (group_sorted['end_time'] - group_sorted['start_time']).dt.total_seconds()
    return group_sorted['total_completion_time']

# Get total completion times for Control and Test groups
control_completion_times = calculate_total_completion_time(control_group)
test_completion_times = calculate_total_completion_time(test_group)

# Step 2: Test for normality using Shapiro-Wilk test (for both groups)
normality_control = stats.shapiro(control_completion_times)
normality_test = stats.shapiro(test_completion_times)

print("Shapiro-Wilk Test for Normality:")
print(f"Control group: W = {normality_control.statistic:.3f}, p-value = {normality_control.pvalue:.3f}")
print(f"Test group: W = {normality_test.statistic:.3f}, p-value = {normality_test.pvalue:.3f}")

# Step 3: Test for equality of variances using Levene's Test
levene_test = stats.levene(control_completion_times, test_completion_times)
print("\nLevene's Test for Equality of Variances:")
print(f"Statistic = {levene_test.statistic:.3f}, p-value = {levene_test.pvalue:.3f}")

# Step 4: Perform an independent samples t-test
t_stat, p_value = stats.ttest_ind(control_completion_times, test_completion_times, equal_var=True)

print("\nIndependent Samples t-Test:")
print(f"t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")

# Step 5: Interpret the result
alpha = 0.05  # significance level
if p_value < alpha:
    print("\nReject the null hypothesis: There is a significant difference in completion times between Control and Test groups.")
else:
    print("\nFail to reject the null hypothesis: There is no significant difference in completion times between Control and Test groups.")

Shapiro-Wilk Test for Normality:
Control group: W = 0.434, p-value = 0.000
Test group: W = 0.470, p-value = 0.000

Levene's Test for Equality of Variances:
Statistic = 387.503, p-value = 0.000

Independent Samples t-Test:
t-statistic = -21.172, p-value = 0.000

Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.


D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 143408.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 177779.
  res = hypotest_fun_out(*samples, **kwds)


In [33]:
import pandas as pd
import scipy.stats as stats

# Ensure 'date_time' is in datetime format
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

# Filter for Control and Test groups
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Function to calculate total completion time for each process_step and group
def calculate_total_completion_time(group):
    group_sorted = group.sort_values(by=['visit_id', 'date_time'])
    group_sorted['start_time'] = group_sorted.groupby('visit_id')['date_time'].transform('min')
    group_sorted['end_time'] = group_sorted.groupby('visit_id')['date_time'].transform('max')
    group_sorted['total_completion_time'] = (group_sorted['end_time'] - group_sorted['start_time']).dt.total_seconds()
    return group_sorted

# Separate completion times for each process_step for Control and Test groups
def perform_hypothesis_tests_by_step(control_group, test_group):
    results = {}

    for process_step in control_group['process_step'].unique():
        # Filter data for each process_step
        control_step = control_group[control_group['process_step'] == process_step]
        test_step = test_group[test_group['process_step'] == process_step]
        
        # Calculate completion times for each group
        control_completion_times = calculate_total_completion_time(control_step)['total_completion_time']
        test_completion_times = calculate_total_completion_time(test_step)['total_completion_time']

        # Shapiro-Wilk test for normality
        normality_control = stats.shapiro(control_completion_times)
        normality_test = stats.shapiro(test_completion_times)

        # Levene's test for equality of variances
        levene_test = stats.levene(control_completion_times, test_completion_times)

        # Independent samples t-test
        t_stat, p_value = stats.ttest_ind(control_completion_times, test_completion_times, equal_var=True)

        # Store results for each process_step
        results[process_step] = {
            "normality_control_p_value": normality_control.pvalue,
            "normality_test_p_value": normality_test.pvalue,
            "levene_test_p_value": levene_test.pvalue,
            "t_statistic": t_stat,
            "p_value": p_value
        }
    
    return results

# Perform hypothesis tests for each process_step
test_results = perform_hypothesis_tests_by_step(control_group, test_group)

# Print the results
for process_step, result in test_results.items():
    print(f"\nResults for Process Step: {process_step}")
    print(f"Shapiro-Wilk Test (Control group) p-value: {result['normality_control_p_value']:.3f}")
    print(f"Shapiro-Wilk Test (Test group) p-value: {result['normality_test_p_value']:.3f}")
    print(f"Levene's Test p-value: {result['levene_test_p_value']:.3f}")
    print(f"t-statistic: {result['t_statistic']:.3f}, t-test p-value: {result['p_value']:.3f}")

    # Interpretation
    if result['p_value'] < 0.05:
        print("Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.")
    else:
        print("Fail to reject the null hypothesis: There is no significant difference in completion times between Control and Test groups.")

D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 17498.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 25716.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 22787.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 25888.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may 


Results for Process Step: confirm
Shapiro-Wilk Test (Control group) p-value: 0.000
Shapiro-Wilk Test (Test group) p-value: 0.000
Levene's Test p-value: 0.000
t-statistic: -16.377, t-test p-value: 0.000
Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.

Results for Process Step: step_3
Shapiro-Wilk Test (Control group) p-value: 0.000
Shapiro-Wilk Test (Test group) p-value: 0.000
Levene's Test p-value: 0.000
t-statistic: -8.000, t-test p-value: 0.000
Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.

Results for Process Step: step_2
Shapiro-Wilk Test (Control group) p-value: 0.000
Shapiro-Wilk Test (Test group) p-value: 0.000
Levene's Test p-value: 0.000
t-statistic: -10.846, t-test p-value: 0.000
Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.

Results for Process Step: step_1
Shapiro-Wilk Te

D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 47589.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 56447.
  res = hypotest_fun_out(*samples, **kwds)


In [36]:
import pandas as pd
import scipy.stats as stats

# Assuming 'clnt_age' is the age column and 'date_time' is already in datetime format

# Define age groups
bins = [0, 30, 40, 50, 100]
labels = ['Under 30', '30-39', '40-49', '50 and above']

# Add a new column for the age group
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels, right=False)

# Filter for Control and Test groups
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Function to calculate total completion time for each process_step, age_group, and variation group
def calculate_total_completion_time(group):
    group_sorted = group.sort_values(by=['visit_id', 'date_time'])
    group_sorted['start_time'] = group_sorted.groupby('visit_id')['date_time'].transform('min')
    group_sorted['end_time'] = group_sorted.groupby('visit_id')['date_time'].transform('max')
    group_sorted['total_completion_time'] = (group_sorted['end_time'] - group_sorted['start_time']).dt.total_seconds()
    return group_sorted

# Perform hypothesis tests for each age group and process_step
def perform_hypothesis_tests_by_age_and_step(control_group, test_group):
    results = {}

    for age_group in control_group['age_group'].unique():
        for process_step in control_group['process_step'].unique():
            # Filter data for each age_group and process_step
            control_step = control_group[(control_group['age_group'] == age_group) & (control_group['process_step'] == process_step)]
            test_step = test_group[(test_group['age_group'] == age_group) & (test_group['process_step'] == process_step)]
            
            # Calculate completion times for each group
            control_completion_times = calculate_total_completion_time(control_step)['total_completion_time']
            test_completion_times = calculate_total_completion_time(test_step)['total_completion_time']

            # Shapiro-Wilk test for normality
            normality_control = stats.shapiro(control_completion_times)
            normality_test = stats.shapiro(test_completion_times)

            # Levene's test for equality of variances
            levene_test = stats.levene(control_completion_times, test_completion_times)

            # Independent samples t-test
            t_stat, p_value = stats.ttest_ind(control_completion_times, test_completion_times, equal_var=True)

            # Store results for each age_group and process_step
            if age_group not in results:
                results[age_group] = {}

            results[age_group][process_step] = {
                "normality_control_p_value": normality_control.pvalue,
                "normality_test_p_value": normality_test.pvalue,
                "levene_test_p_value": levene_test.pvalue,
                "t_statistic": t_stat,
                "p_value": p_value
            }
    
    return results

# Perform hypothesis tests for each age group and process_step
test_results = perform_hypothesis_tests_by_age_and_step(control_group, test_group)

# Print the results
for age_group, process_results in test_results.items():
    print(f"\nResults for Age Group: {age_group}")
    for process_step, result in process_results.items():
        print(f"\n  Process Step: {process_step}")
        print(f"    Shapiro-Wilk Test (Control group) p-value: {result['normality_control_p_value']:.3f}")
        print(f"    Shapiro-Wilk Test (Test group) p-value: {result['normality_test_p_value']:.3f}")
        print(f"    Levene's Test p-value: {result['levene_test_p_value']:.3f}")
        print(f"    t-statistic: {result['t_statistic']:.3f}, t-test p-value: {result['p_value']:.3f}")

        # Interpretation
        if result['p_value'] < 0.05:
            print("    Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.")
        else:
            print("    Fail to reject the null hypothesis: There is no significant difference in completion times between Control and Test groups.")

D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 8455.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 12451.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 11168.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 13256.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may n


Results for Age Group: 50 and above

  Process Step: confirm
    Shapiro-Wilk Test (Control group) p-value: 0.000
    Shapiro-Wilk Test (Test group) p-value: 0.000
    Levene's Test p-value: 0.000
    t-statistic: -11.702, t-test p-value: 0.000
    Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.

  Process Step: step_3
    Shapiro-Wilk Test (Control group) p-value: 0.000
    Shapiro-Wilk Test (Test group) p-value: 0.000
    Levene's Test p-value: 0.000
    t-statistic: -8.848, t-test p-value: 0.000
    Reject the null hypothesis: There is a significant difference in completion times between Control and Test groups.

  Process Step: step_2
    Shapiro-Wilk Test (Control group) p-value: 0.000
    Shapiro-Wilk Test (Test group) p-value: 0.000
    Levene's Test p-value: 0.000
    t-statistic: -12.732, t-test p-value: 0.000
    Reject the null hypothesis: There is a significant difference in completion times between Control

D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6088.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 7110.
  res = hypotest_fun_out(*samples, **kwds)
D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 8311.
  res = hypotest_fun_out(*samples, **kwds)
